In [11]:
# 단계 1: 폰트 설치
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null
#fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

#font = fm.FontProperties(fname=fontpath, size=9)

#fm._rebuild()



fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                         # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설치

In [ ]:
# 단계 2: 런타임 재시작
import os
os.kill(os.getpid(), 9)

In [40]:
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) # 폰트 설

In [15]:
!pip install torch torchvision
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


In [85]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



# CSV 파일 경로
file_path0 = "/content/예매.csv"
#file_path0 = "/content/SAC_ADAVANTK_DATA_202312.csv"

# 파일 읽기
#df0 = pd.read_csv(file_path0, encoding='euc-kr')
try:
    df = pd.read_csv(file_path0, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path0, encoding='ISO-8859-1')

print(df.head())
print(df.count())

                   PBLPRFR_NM PBLPRFR_PLACE_NM  PBLPRFR_DE  PBLPRFR_TIME  \
0  국립합창단 창단 50주년 기념 <한국합창교향곡>             콘서트홀    20230321        193000   
1  국립합창단 창단 50주년 기념 <한국합창교향곡>             콘서트홀    20230321        193000   
2  국립합창단 창단 50주년 기념 <한국합창교향곡>             콘서트홀    20230321        193000   
3  국립합창단 창단 50주년 기념 <한국합창교향곡>             콘서트홀    20230321        193000   
4  국립합창단 창단 50주년 기념 <한국합창교향곡>             콘서트홀    20230321        193000   

  GENRE_NM                             PRC_CN  \
0       합창  R석 30000원 / S석 20000원 / A석 10000원   
1       합창  R석 30000원 / S석 20000원 / A석 10000원   
2       합창  R석 30000원 / S석 20000원 / A석 10000원   
3       합창  R석 30000원 / S석 20000원 / A석 10000원   
4       합창  R석 30000원 / S석 20000원 / A석 10000원   

                                            HMPG_URL  SETLE_DE  SETLE_TIME  \
0  https://www.sac.or.kr/site/main/show/show_view...  20230308       70000   
1  https://www.sac.or.kr/site/main/show/show_view...  20230221      190000   
2  https://www

In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import joblib  # 모델 저장을 위한 라이브러리

# 데이터 로드
df = pd.read_csv(file_path0, encoding='utf-8')

# NaN 값을 포함하는 행 제거
df.dropna(inplace=True)

# 타겟 변수 레이블 인코딩
label_encoder = LabelEncoder()
df['GENRE_NM'] = label_encoder.fit_transform(df['GENRE_NM'])

# object형 열만 선택
object_cols = df.select_dtypes(include=['object']).columns

# 원-핫 인코딩 수행 (타겟 변수를 제외하고 인코딩)
df_encoded = pd.get_dummies(df, columns=object_cols, drop_first=True)

# 입력 변수와 타겟 변수 분리
X = df_encoded.drop('GENRE_NM', axis=1)  # 타겟 변수
y = df_encoded['GENRE_NM']  # 인코딩된 타겟 변수

# 상관관계 분석 및 낮은 상관관계 특성 제거
correlation_matrix = X.corr()
# 상관계수가 0.1 이하인 특성 제거
low_corr_features = correlation_matrix.columns[correlation_matrix.abs().lt(0.1).any()]
X = X.drop(low_corr_features, axis=1)

# K-겹 교차 검증 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 신경망 모델 정의
class GenrePredictionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(GenrePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# 하이퍼파라미터 설정
num_classes = len(label_encoder.classes_)
input_dim = X.shape[1]
learning_rate = 0.0005
num_epochs = 50
patience = 5  # 얼리스탑을 위한 인내값

# K-겹 교차 검증
best_accuracy = 0.0
best_model = None

for fold, (train_index, val_index) in enumerate(kf.split(X)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # PyTorch 데이터셋 클래스 정의
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train.values, dtype=torch.float32),
                                                   torch.tensor(y_train.values, dtype=torch.long))
    val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val.values, dtype=torch.float32),
                                                 torch.tensor(y_val.values, dtype=torch.long))

    # DataLoader 생성
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

    # 모델 초기화
    model = GenrePredictionModel(input_dim, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 얼리스탑 변수 초기화
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        for features, targets in train_loader:
            # Forward Pass
            outputs = model(features)
            loss = criterion(outputs, targets)

            # Backward Pass and Optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation step
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for features, targets in val_loader:
                outputs = model(features)
                val_loss += criterion(outputs, targets).item()
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == targets).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = val_correct / len(val_dataset)

        print(f'Fold [{fold + 1}], Epoch [{epoch + 1}/{num_epochs}], Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # 얼리스탑
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_accuracy = val_accuracy
            epochs_no_improve = 0
            best_model = model.state_dict()  # 모델의 가중치를 저장
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping triggered.")
                break

# 최적의 모델 저장
if best_model is not None:
    torch.save(best_model, 'best_genre_prediction_model.pth')
    print("최적의 모델이 저장되었습니다.")

print(f'최고 검증 정확도: {best_accuracy:.4f}')


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Fold [1], Epoch [1/50], Val Loss: 1.7313, Val Accuracy: 0.3009
Fold [1], Epoch [2/50], Val Loss: 1.7301, Val Accuracy: 0.3009
Fold [1], Epoch [3/50], Val Loss: 1.7301, Val Accuracy: 0.3009
Fold [1], Epoch [4/50], Val Loss: 1.7303, Val Accuracy: 0.3009
Fold [1], Epoch [5/50], Val Loss: 1.7305, Val Accuracy: 0.3009
Fold [1], Epoch [6/50], Val Loss: 1.7307, Val Accuracy: 0.3009
Fold [1], Epoch [7/50], Val Loss: 1.7302, Val Accuracy: 0.3009
Early stopping triggered.
Fold [2], Epoch [1/50], Val Loss: 1.7187, Val Accuracy: 0.3102
Fold [2], Epoch [2/50], Val Loss: 1.7186, Val Accuracy: 0.3102
Fold [2], Epoch [3/50], Val Loss: 1.7183, Val Accuracy: 0.3102
Fold [2], Epoch [4/50], Val Loss: 1.7183, Val Accuracy: 0.3102
Fold [2], Epoch [5/50], Val Loss: 1.7180, Val Accuracy: 0.3102
Fold [2], Epoch [6/50], Val Loss: 1.7178, Val Accuracy: 0.3102
Fold [2], Epoch [7/50], Val Loss: 1.7179, Val Accuracy: 0.3102
Fold [2], Epoch [8/50], Val Loss: 1.7180, Val Accuracy: 0.3102
Fold [2], Epoch [9/50], Val L

In [97]:
class ImprovedGenrePredictionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ImprovedGenrePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # 노드 수 증가
        self.bn1 = nn.BatchNorm1d(256)  # 배치 정규화 추가
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 128)  # 노드 수 증가
        self.bn2 = nn.BatchNorm1d(128)  # 배치 정규화 추가
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # 배치 정규화 적용
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))  # 배치 정규화 적용
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# 모델 초기화
model = ImprovedGenrePredictionModel(input_dim, num_classes)

# 학습률 스케줄러 추가
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

# 모델 학습 코드에서 각 에포크 후 스케줄러 업데이트
for epoch in range(num_epochs):
    # ... (훈련 코드)

    # Validation step
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for features, targets in val_loader:
            outputs = model(features)
            val_loss += criterion(outputs, targets).item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_loss /= len(val_loader)
    scheduler.step(val_loss)  # Validation loss에 따라 학습률 조정


In [96]:
class GenrePredictionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(GenrePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 초기화
input_dim = X_train.shape[1]
num_classes = len(label_encoder.classes_)
model = GenrePredictionModel(input_dim, num_classes)


In [98]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [99]:
num_epochs = 50

for epoch in range(num_epochs):
    for features, targets in train_loader:
        # Forward Pass
        outputs = model(features)
        loss = criterion(outputs, targets)

        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/50], Loss: 1.9234
Epoch [20/50], Loss: 1.8437
Epoch [30/50], Loss: 1.6316
Epoch [40/50], Loss: 1.7446
Epoch [50/50], Loss: 1.4532


In [100]:
import numpy as np

# 모델 평가
model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []  # 실제 정답을 저장할 리스트 추가
    for features, targets in test_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())
        y_true.extend(targets.numpy())  # 실제 정답 저장

# 결과 확인
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_true_labels = label_encoder.inverse_transform(y_true)  # 실제 정답 레이블 변환

# 정확도 계산
accuracy = np.mean(np.array(y_pred) == np.array(y_true))  # NumPy 배열로 변환 후 비교
print("예측된 장르:", y_pred_labels)
print("실제 장르:", y_true_labels)
print(f'모델 정확도: {accuracy:.4f}')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x205 and 0x256)

In [7]:
# 환불되지 않은 티켓의 수를 세어 관람객 수를 계산
df['관람객 수'] = df[df['RFN_AT'] != 'Y'].groupby('PBLPRFR_NM')['SETLE_PRICE'].transform('count')

# PBLPRFR_NM을 기준으로 중복된 행을 제거하고, 첫 번째 관람객 수만 남기기
unique_audience_counts = df[['PBLPRFR_NM', '관람객 수']].drop_duplicates(subset='PBLPRFR_NM')

# 결과 확인
print(unique_audience_counts)


#print(df[['PBLPRFR_NM', '관람객 수']])

                             PBLPRFR_NM   관람객 수
0            국립합창단 창단 50주년 기념 <한국합창교향곡>   487.0
597                       이창형 더블베이스 독주회    33.0
635                     연극 <셰익스피어 인 러브>  7878.0
830               서울 심포닉 윈드 오케스트라 창단연주회    87.0
937       뮤지컬 < 베토벤; Beethoven Secret >     NaN
...                                 ...     ...
1046719    서울 리조이스 앙상블 스페셜 콘서트 ‘위로와 희망’     2.0
1046920                     최재원 피아노 독주회     8.0
1046928                     최승용 비올라 독주회    41.0
1046979                      유니버설발레단 지젤   716.0
1047795  제 3회 어텀 실내악 페스티벌 2021 - Part.3     NaN

[2299 rows x 2 columns]


In [8]:
# '관람객 수' 컬럼의 Null 값 개수 확인
null_count = df['관람객 수'].isnull().sum()

# Null 값이 있는 행 확인
null_rows = df[df['관람객 수'].isnull()]

# 결과 출력
print(f"'관람객 수' 컬럼의 Null 값 개수: {null_count}")
print("Null 값이 있는 행:")
print(null_rows[['PBLPRFR_NM', '관람객 수']])


'관람객 수' 컬럼의 Null 값 개수: 447303
Null 값이 있는 행:
                         PBLPRFR_NM  관람객 수
6        국립합창단 창단 50주년 기념 <한국합창교향곡>    NaN
20       국립합창단 창단 50주년 기념 <한국합창교향곡>    NaN
22       국립합창단 창단 50주년 기념 <한국합창교향곡>    NaN
26       국립합창단 창단 50주년 기념 <한국합창교향곡>    NaN
27       국립합창단 창단 50주년 기념 <한국합창교향곡>    NaN
...                             ...    ...
1048562                  유니버설발레단 지젤    NaN
1048563                  유니버설발레단 지젤    NaN
1048564                  유니버설발레단 지젤    NaN
1048567                  유니버설발레단 지젤    NaN
1048569                  유니버설발레단 지젤    NaN

[447303 rows x 2 columns]


In [32]:
# '관람객 수'가 Null이 아닌 행 중에서 PBLPRFR_NM 중복을 제거
new_df = df[df['관람객 수'].notnull()].drop_duplicates(subset='PBLPRFR_NM').copy()

# 결과 확인
print(new_df[['PBLPRFR_NM', '관람객 수']].head())

row_count = new_df.shape[0]
print(f"새로운 데이터프레임의 행 개수: {row_count}")

                        PBLPRFR_NM    관람객 수
0       국립합창단 창단 50주년 기념 <한국합창교향곡>    487.0
597                  이창형 더블베이스 독주회     33.0
635                연극 <셰익스피어 인 러브>   7878.0
830          서울 심포닉 윈드 오케스트라 창단연주회     87.0
938  뮤지컬 < 베토벤; Beethoven Secret >  21952.0
새로운 데이터프레임의 행 개수: 2286


In [33]:
# 공연 수익 계산
new_df['공연 수익'] = new_df['관람객 수'] * new_df['SETLE_PRICE']

# 결과 확인
print(new_df[['PBLPRFR_NM', '관람객 수', 'SETLE_PRICE', '공연 수익']].head())


                        PBLPRFR_NM    관람객 수  SETLE_PRICE         공연 수익
0       국립합창단 창단 50주년 기념 <한국합창교향곡>    487.0        24000  1.168800e+07
597                  이창형 더블베이스 독주회     33.0        24000  7.920000e+05
635                연극 <셰익스피어 인 러브>   7878.0        93500  7.365930e+08
830          서울 심포닉 윈드 오케스트라 창단연주회     87.0         8000  6.960000e+05
938  뮤지컬 < 베토벤; Beethoven Secret >  21952.0       161500  3.545248e+09


In [38]:
# 공연 수익을 만 단위로 포맷팅
new_df['공연 수익'] = (new_df['공연 수익'] / 10000).apply(lambda x: f"{x:,.0f} 만원")

# 결과 확인
print(new_df[['PBLPRFR_NM', '관람객 수', 'SETLE_PRICE', '공연 수익']].head())



TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [39]:
print(new_df['공연 수익'])

0            1,169 만원
597             79 만원
635         73,659 만원
830             70 만원
938        354,525 만원
              ...    
1046719          4 만원
1046920         19 만원
1046928         86 만원
1046979      1,504 만원
1047796        506 만원
Name: 공연 수익, Length: 2286, dtype: object


In [36]:
# 공연 수익 계산
df['공연 수익'] = df['관람객 수'] * df['SETLE_PRICE']

# 특성과 타겟 변수 선택
features = ['PBLPRFR_PLACE_NM', 'GENRE_NM', 'DSCNT_CN', 'PBLPRFR_DE', '관람객 수']
target = '공연 수익'

# 특성과 타겟 변수 분리
X = df[features]
y = df[target]

# 레이블 인코딩
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))
    label_encoders[column] = le

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

<ipython-input-36-bca879bc7446>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column].astype(str))
<ipython-input-36-bca879bc7446>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column].astype(str))
<ipython-input-36-bca879bc7446>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

class RevenueDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.X = torch.tensor(features, dtype=torch.float32)
        self.y = torch.tensor(targets.values, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 데이터셋 생성
train_dataset = RevenueDataset(X_train, y_train)
test_dataset = RevenueDataset(X_test, y_test)

# DataLoader 생성 (배치 크기를 64로 조정)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [27]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

# 모델 초기화
input_dim = X_train.shape[1]
model = LinearRegressionModel(input_dim)

In [28]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

num_epochs = 20  # 에포크 수 조정
patience = 5  # 조기 종료를 위한 인내 기간
best_loss = float('inf')
patience_counter = 0


In [29]:
for epoch in range(num_epochs):
    for features, targets in train_loader:
        # Forward Pass
        outputs = model(features).squeeze()
        loss = criterion(outputs, targets)

        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크마다 손실 확인
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 조기 종료 로직
    if loss.item() < best_loss:
        best_loss = loss.item()
        patience_counter = 0  # 손실 개선 시 카운터 초기화
    else:
        patience_counter += 1  # 손실 개선이 없으면 카운터 증가

    if patience_counter >= patience:
        print(f'조기 종료: {epoch + 1} 에포크에서 훈련 종료')
        break

조기 종료: 5 에포크에서 훈련 종료


In [30]:
# 모델 평가
model.eval()
with torch.no_grad():
    y_pred = []
    for features, targets in test_loader:
        outputs = model(features).squeeze()
        y_pred.extend(outputs.numpy())

# 결과 확인
y_pred = pd.Series(y_pred)
print("예측된 공연 수익:", y_pred.describe())

예측된 공연 수익: count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
dtype: float64
